# Modelling

#### Идеи, которые мы тестировали в рамках настройки модели 

Часть из них вошли в итоговое решение, некоторые были отвергнуты, так как не дали ощутимого прироста в метриках

Обогащение данных сторонними датасетами (население и средняя заработная плата по регионам)

In [ ]:
city_population = pd.read_csv('city_population.csv')
zarplaty = pd.read_excel('zarplaty.xlsx')

city_population_clean = city_population.groupby('settlement').agg({'population':'sum'}).reset_index()
city_population_clean.columns = ['city', 'city_population']
city_population_clean['city_population']
city_population_clean.city = city_population_clean.city.apply(lambda x: x.lower())
df = df.merge(city_population_clean, on = 'city', how='left')
     
df = df.merge(zarplaty, on = 'region', how='left')
df['zarplata'] = pd.to_numeric(df['zarplata'], downcast = 'unsigned')
df['floor_type'] = df['floor'].apply(lambda x: floor_type(x))

Группировка населенных пунктов по численности населения и выделение столиц

In [ ]:
def city_type(row):
    if row >=1000000:
        return "1Million"
    elif  (row<1000000)&(row >200000):
        return "Medium"
    elif  (row <=200000):
        return "Small"

df['city_type'] = df['city_population'].apply(lambda x: city_type(x))
df.loc[df.city  == 'москва', 'city_type'] = "Capital"
df.loc[df.city  == 'санкт-Петербург', 'city_type'] = "Capital"

Преобразование строковых значений этажей в категории 0 и 1 

In [ ]:
def floor_type(row):
    if ('1' in str(row))&(row!=-1):
        return 1
    else:
        return 0

Добавление возраста построек в качестве нового признака

In [ ]:
df['age'] = round(2021 - df['reform_mean_year_building_500'])

Сокращение широты и долготы и создание нового признака путем объединения через “_”

In [ ]:
i = 2
data['lat_'+str(i)] = data['lat'].round(i)
data['lng_'+str(i)] = data['lng'].round(i)
data['square_ll'+str(i)] = data['lat_'+str(i)].astype(str) + '_' + data['lng_'+str(i)].astype(str)

Группировка по типам недвижимости и расчет средней цены квадратного метра по группе в качестве нового признака

In [ ]:
temp = (
    train_data.groupby('realty_type')['per_square_meter_price'].agg(['mean'])
    .rename({'mean':'per_square_meter_price_mean'},axis=1)
)

train_data = pd.merge(train_data, temp, on='realty_type',how='left')
test_data = pd.merge(test_data, temp, on='realty_type',how='left')

Cоздание нового признака “среднее количество объектов на 100 м” из признаков типа “количество объектов в радиусе 1 км”, “в радиусе 750 м”, в радиусе “500 м” и удаление “лишних” признаков - для признака с высоким коэффициентом корреляции с целевой переменной

In [ ]:
def catering_per_100m(df):
    df['catering_per_100m'] = (
        (df['osm_catering_points_in_0.005'] / 5 
         + df['osm_catering_points_in_0.0075'] / 7.5 
         + df['osm_catering_points_in_0.01'] / 10 
         + df['osm_catering_points_in_0.001'] / 1) 
        / 4
    )

df = df.drop(
    [
        'osm_catering_points_in_0.005', 
     'osm_catering_points_in_0.0075',
     'osm_catering_points_in_0.01',
     'osm_catering_points_in_0.001'
    ], axis = 1
)

train_data = catering_per_100m(train_data)
test_data = catering_per_100m(test_data)

Работа с выбросами в предсказанных ценах (“подрезка предикта”)

In [ ]:
output = pd.DataFrame({'id': test_data['id'],
                       'per_square_meter_price': automl.predict(test_data).data[:, 0]})
output['per_square_meter_price'] = output['per_square_meter_price'] * 0.9

output.loc[output['per_square_meter_price'] >= 200000, 'per_square_meter_price'] \
    = output.loc[output['per_square_meter_price'] >= 200000, 'per_square_meter_price'] * 0.9